In [81]:
from Crypto.Util import number
import hashlib
import math 

## Issuer

In [82]:
# setting up the encryption
bitsize = 32
p = number.getPrime(bitsize)
q = number.getPrime(bitsize)

# The public parts
n = p*q
nsize = 2*bitsize
msize = nsize
# Generate quadratic residues mod n
S = pow(number.getRandomRange(2, n), 2, n)
Z = pow(number.getRandomRange(2, n), 2, n)


print(f'Public key is:\nn = {n}\nS = {S}\nZ = {Z}')
print(f'\nPrivate key is: {(p,q)}')

Public key is:
n = 7684346552150936617
S = 4322715710146621019
Z = 4067608610903955793

Private key is: (3394656751, 2263659367)


## Holder

In [83]:
# this hash is not how you should do, just for demo
attr = number.getRandomNBitInteger(nsize)
# calculate hash for all attributes
c_hash = hashlib.sha256(str(attr).encode('utf-8')).hexdigest()
c = int(c_hash, base=32)

# Age message
M = 35

print(f'Message is:\nM = {M}')
print(f'\nHash c is: {c}')

Message is:
M = 35

Hash c is: 757798271031644061719415919146725901054370398843449034446892068220527165546072968857026517369989


Message is encrypted via

1. Generate random integer $\bar{m_i}$

2. Compute
$$
\hat{m_i} = \bar{m_i} + Dm_i
$$

In [84]:
M_bar = number.getRandomNBitInteger(nsize)
M_hat = M_bar + c*M

print(f'Encrypted message is:\nM_bar = {M_bar}\nM_hat = {M_hat}')

Encrypted message is:
M_bar = 16279841402853139494
M_hat = 26522939486107542160179557170135406536902963959520716205641222387718450794112570189837330961089109


# Check predicates such as "A is older than 18 years old".

In [85]:
# This is done for each predicate
b = 1
ap1 = 18
mp1 = M
mp1_bar = M_bar
mp1_hat = M_hat


#### Lagrange’s Four Square Theorem
Lagrange’s Four Square Theorem states that every natural number can be written as sum of squares of four non negative integers.


$$
\Delta \leftarrow u_1^2 + u_2^2 +u_3^2 + u_4^2
$$

Note that ${\Delta}$ will always be non-negative if the predicate is true

In [86]:
def largest_square_less_than(delta):
    return int(math.floor(math.sqrt(float(delta))))


def four_squares(delta):
    d = int(delta)
    u1 = largest_square_less_than(d)
    u2 = largest_square_less_than(d - math.pow(u1, 2))
    u3 = largest_square_less_than(d - math.pow(u1, 2) - math.pow(u2, 2))
    u4 = largest_square_less_than(d - math.pow(u1, 2) - math.pow(u2, 2) - math.pow(u3, 2))

    uPow =  int(math.pow(u1, 2) + math.pow(u2, 2) + math.pow(u3, 2) + math.pow(u4, 2))
    if uPow == d:
        roots = [u1, u2, u3, u4]
    else:
        roots = [-1, -1, -1, -1]

    return roots

In [87]:
# mp_int = int(number.long_to_bytes(M).decode('utf-8'))
mp_int = M
delta = mp_int - ap1

# check delta is greater than > 0
assert(delta > 0)

roots = four_squares(delta)
# check delta is greater than > 0
assert(sum(roots) > 0)

print(f"Delta = {delta},\nRoots: (u_1, u_2, u_3, u_4) = {(roots[0], roots[1], roots[2], roots[3])}")

Delta = 17,
Roots: (u_1, u_2, u_3, u_4) = (4, 1, 0, 0)


Calculate $T$ equations

1. Generate random integer numbers $r_1, r_2, r_3, r_4, r_{\Delta}$
2. Compute 
$$ T_1 \leftarrow Z^{u_1}S^{r_1}\pmod{n} $$
$$ T_2 \leftarrow Z^{u_2}S^{r_2}\pmod{n} $$
$$ T_3 \leftarrow Z^{u_3}S^{r_3}\pmod{n} $$
$$ T_4 \leftarrow Z^{u_4}S^{r_4}\pmod{n} $$
$$ T_{\Delta} \leftarrow  Z^{\Delta}S^{r_{\Delta}} \pmod{n} $$

and add these values to $\mathcal{C}$ in the order ${T_1},{T_2},{T_3},{T_4},{T_{\Delta}}$.


In [88]:
# Generate random number
r1 = number.getRandomNBitInteger(msize*nsize + 1)
r2 = number.getRandomNBitInteger(msize*nsize + 1)
r3 = number.getRandomNBitInteger(msize*nsize + 1)
r4 = number.getRandomNBitInteger(msize*nsize + 1)
rd = number.getRandomNBitInteger(msize*nsize + 1)

# Compute T equations
T1 = pow(Z, roots[0], n) * pow(S, r1, n)
T2 = pow(Z, roots[1], n) * pow(S, r2, n)
T3 = pow(Z, roots[2], n) * pow(S, r3, n)
T4 = pow(Z, roots[3], n) * pow(S, r4, n)
Td = pow(Z, delta, n) * pow(S, rd, n)

print(f"Predicate C is:\nT_1 = {T1}\nT_2 = {T2}\nT_3 = {T3}\nT_4 = {T4}\nT_Delta = {Td}")

Predicate C is:
T_1 = 1355431756830407573663939491211849235
T_2 = 15798092764946956752310386941296242913
T_3 = 2662576874584118566
T_4 = 4787823244777280787
T_Delta = 26096299314051644520126597839691242846


Calculate $\bar{T}$ and $\bar{K}$ equations

1. Generate random integer numbers $\bar{u_1},\bar{u_2},\bar{u_3},\bar{u_4}$
2. Generate random integer numbers $\bar{r_1},\bar{r_2},\bar{r_3},\bar{r_4},\bar{r_{\Delta}}$
3. Compute 
$$ \bar{T_1} \leftarrow Z^{\bar{u_1}}S^{\bar{r_1}}\pmod{n} $$
$$ \bar{T_2} \leftarrow Z^{\bar{u_2}}S^{\bar{r_2}}\pmod{n} $$
$$ \bar{T_3} \leftarrow Z^{\bar{u_3}}S^{\bar{r_3}}\pmod{n} $$
$$ \bar{T_4} \leftarrow Z^{\bar{u_4}}S^{\bar{r_4}}\pmod{n} $$
$$ \bar{T_{\Delta}} \leftarrow  Z^{\bar{m_j}}(S^{b})^{\bar{r_{\Delta}}} \pmod{n} $$

and add this values to $\mathcal{T}$ in the order $\bar{T_1},\bar{T_2},\bar{T_3},\bar{T_4}, \bar{T_{\Delta}}$.

Where $\bar{m_j}$ is from the earlier generated random number.

4. Generate random integer numbers $\bar{\alpha}$
5. Compute
$$
\bar{K} \leftarrow S^{\bar{\alpha}}T_1^{\bar{u_1}}T_2^{\bar{u_2}}
T_3^{\bar{u_3}} T_4^{\bar{u_4}}\pmod{n}
$$
Add $\bar{K}$ to $\mathcal{T}$.

In [89]:
# Generate random number
u1_bar = number.getRandomNBitInteger(nsize + 1)
u2_bar = number.getRandomNBitInteger(nsize + 1)
u3_bar = number.getRandomNBitInteger(nsize + 1)
u4_bar = number.getRandomNBitInteger(nsize + 1)

r1_bar = number.getRandomNBitInteger(nsize + 1)
r2_bar = number.getRandomNBitInteger(nsize + 1)
r3_bar = number.getRandomNBitInteger(nsize + 1)
r4_bar = number.getRandomNBitInteger(nsize + 1)
rd_bar = number.getRandomNBitInteger(nsize + 1)

# Compute T_bar equations
T1_bar = pow(Z, u1_bar, n) * pow(S, r1_bar, n) % n
T2_bar = pow(Z, u2_bar, n) * pow(S, r2_bar, n) % n
T3_bar = pow(Z, u3_bar, n) * pow(S, r3_bar, n) % n
T4_bar = pow(Z, u4_bar, n) * pow(S, r4_bar, n) % n
Td_bar = pow(Z, mp1_bar, n) * pow(S, b*rd_bar, n) % n

# Compute K_bar
alpha_bar = number.getRandomNBitInteger(msize*nsize + 1)
K_bar = pow(S, alpha_bar, n) * pow(T1, u1_bar, n) * pow(T2, u2_bar, n) * pow(T3, u3_bar, n) * pow(T4, u4_bar, n) % n


print(f"Proof predicate is:\nT1_bar = {T1_bar}\nT2_bar = {T2_bar}\nT3_bar = {T3_bar}\nT4_bar = {T4_bar}\nTd_bar = {Td_bar}\nalpha_bar = {alpha_bar}\nK_bar = {K_bar}")


Proof predicate is:
T1_bar = 438305240496466990
T2_bar = 3728270313340511770
T3_bar = 4976719232043502788
T4_bar = 4848288195323148341
Td_bar = 4958529726905195168
alpha_bar = 20351931673511362676454767853627371163400905471108211604499680836262910399647485616490366042752299586784930566836208912082329999153077299089606374936593760447627889879585561932735745583276975892303756603682978258958696504301462618167215942726340203817426932147253721035189463458452953713496570474237091245273761617001704636601654887475271843231639859290234453159489990763582609682201080529911884441101022691788624442135964689943038254978445885778203322085887467044386729862373881644895526536707893569162590505605486604590952620492099493118974676803364162271979221868049401847992071969306460998384647809223544701541052986204959685965875352840080677902759198455594851594031298916179449930378353798505606042563339403318985169611878590016529637937379171850978028150241703176324950865000956704900026250547158006992599950614454708

For each predicate $p$ compute:

1. For {$i \in {1, 2, 3, 4}$}, compute
\begin{align*}
\hat{u_i} \leftarrow \bar{u_i}+Du_i \\
\hat{r_i} \leftarrow \bar{r_i}+Dr_i \\
\end{align*}

2. Compute
$$
\hat{r_{\Delta}} \leftarrow \bar{r_{\Delta}}+Dr_{\Delta}
$$

3. Compute
$$
{\alpha} \leftarrow r_{\Delta}- u_1r_1 - u_2r_2 - u_3r_3 - u_4r_4
$$

4. Compute
$$
\hat{\alpha} \leftarrow \bar{\alpha}+D\alpha
$$


In [90]:
u1_hat = u1_bar + c*roots[0]
u2_hat = u2_bar + c*roots[1]
u3_hat = u3_bar + c*roots[2]
u4_hat = u4_bar + c*roots[3]

r1_hat = r1_bar + c*r1
r2_hat = r2_bar + c*r2
r3_hat = r3_bar + c*r3
r4_hat = r4_bar + c*r4

rd_hat = rd_bar + c*rd

alpha = rd - roots[0]*r1 - roots[1]*r2 - roots[2]*r3 - roots[3]*r4
alpha_hat = alpha_bar + c*alpha

# verify
assert(pow(T1, u1_hat, n) == pow(T1, u1_bar, n) * pow(T1, c*roots[0], n) % n)
assert(pow(T2, u2_hat, n) == pow(T2, u2_bar, n) * pow(T2, c*roots[1], n) % n)
assert(pow(T3, u3_hat, n) == pow(T3, u3_bar, n) * pow(T3, c*roots[2], n) % n)
assert(pow(T4, u4_hat, n) == pow(T4, u4_bar, n) * pow(T4, c*roots[3], n) % n)

### Holder sent proof to Verifier

The values $Pr_p =( \{\hat{u_i}\}, \{\hat{r_i}\},\hat{r_{\Delta}},\hat{\alpha},\hat{m_j})$ are the sub-proof for predicate $p$.

The values $C=( \{{T_i}\}, {T_\Delta},)$ are the proof for credential $C$.

Then $(c,\{Pr_p\},\mathcal{C})$ is sent to the Verifier as proof


For the predicate "A is older than 18 years old" this values are sent:

In [91]:
print(f"T_1 = {T1}\nT_2 = {T2}\nT_3 = {T3}\nT_4 = {T4}\nT_Delta = {Td}")
print(f"(u1_hat, u2_hat, u3_hat, u4_hat) = {(u1_hat, u2_hat, u3_hat, u4_hat)}")
print(f"(r1_hat, r2_hat, r3_hat, r4_hat) = {(r1_hat, r2_hat, r3_hat, r4_hat)}")
print(f"rd_hat = {rd_hat}\nalpha_hat = {alpha_hat}\nM_hat = {M_hat}")

T_1 = 1355431756830407573663939491211849235
T_2 = 15798092764946956752310386941296242913
T_3 = 2662576874584118566
T_4 = 4787823244777280787
T_Delta = 26096299314051644520126597839691242846
(u1_hat, u2_hat, u3_hat, u4_hat) = (3031193084126576246877663676586903604217481595373796137787568272882108662184327027684635016406348, 757798271031644061719415919146725901054370398843449034446892068220527165546103842451318174208644, 25642741881011862137, 29543807907252906501)
(r1_hat, r2_hat, r3_hat, r4_hat) = (10767878755456668261361249083285912696714276124703978161808376154522110943313265218646333540416252029326817227941659398124736775333791928385071441471931155877059519514380569720583235569959170828844720752881939238555190876756034995140457930323130023185019604709700555422227344171827370222315839335238577274552421524667982654965434891817178632386795313688365259231799976133198705909495224336545450529611832574768595967937707703749214447441027262555389931815629531807726055048859363333608819145460943

## Verifier validate predicate

Using $Pr_p$ and $\mathcal{C}$ compute 

1. $\text{For }1\leq i \leq 4$, compute
$$
\hat{T_i} \leftarrow T_i^{-D}Z^{\hat{u_i}} S^{\hat{r_i}}\pmod{n};
$$

2. Compute
$$
\hat{T_{\Delta}} \leftarrow \left((T_{\Delta})^bZ^{\Delta'}\right)^{-D}Z^{\hat{m_j}}(S^b)^{\hat{r_{\Delta}}}\pmod{n};
$$

3. Compute
$$
\hat{K}\leftarrow S^{\hat{\alpha}}(T_{\Delta})^{-D}T_1^{\hat{u_1}}T_2^{\hat{u_2}}T_3^{\hat{u_3}}T_4^{\hat{u_4}}
\pmod{n},
$$
and add these values to  $\hat{\mathcal{T}}$ in the order $\hat{T_1},\hat{T_2} ,\hat{T_3},\hat{T_4},\hat{T_{\Delta}},\hat{K}    $.

In [92]:
delta_prim = ap1

# Compute T_hat equations
T1_hat = pow(T1, -c, n) * pow(Z, u1_hat, n) * pow(S, r1_hat, n) % n
T2_hat = pow(T2, -c, n) * pow(Z, u2_hat, n) * pow(S, r2_hat, n) % n
T3_hat = pow(T3, -c, n) * pow(Z, u3_hat, n) * pow(S, r3_hat, n) % n
T4_hat = pow(T4, -c, n) * pow(Z, u4_hat, n) * pow(S, r4_hat, n) % n
Td_hat = pow(Td, -b*c, n) * pow(Z, -delta_prim*c, n) *  pow(Z, mp1_hat, n) * pow(S, b * rd_hat, n)  % n


# Compute K_hat
K_hat = pow(S, alpha_hat, n) * pow(Td, -c, n) * pow(T1, u1_hat, n) * pow(T2, u2_hat, n) * pow(T3, u3_hat, n) * pow(T4, u4_hat, n) % n

### Equality check

In [93]:
tau = [T1_bar, T2_bar, T3_bar, T4_bar, Td_bar, K_bar]
tau_hat = [T1_hat, T2_hat, T3_hat, T4_hat, Td_hat, K_hat]

assert(tau_hat == tau)

In [94]:
print('The equality has been VERIFIED when this line is printed!')

The equality has been VERIFIED when this line is printed!
